<a href="https://colab.research.google.com/github/AimanFariz/Sports-Analytics/blob/main/Central_Defenders_Generalized_Weighted_Attribute_Rating.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install mplsoccer

In [2]:
import pandas as pd
import numpy as np
from scipy.stats import zscore
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir("{PATH TO THE FILES}")

#Step 1: Read Data
# df = pd.read_csv("CSL 2024 CB.csv")
df = pd.read_csv("MLS 2024 CB.csv")
# df = pd.read_csv("CB mixed.csv")
df.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,Player,Team,Team within selected timeframe,Position,Age,Market value,Contract expires,Matches played,Minutes played,Goals,...,Prevented goals per 90,Back passes received as GK per 90,Exits per 90,Aerial duels per 90.1,Free kicks per 90,Direct free kicks per 90,"Direct free kicks on target, %",Corners per 90,Penalties taken,"Penalty conversion, %"
0,Marlon,Los Angeles FC,Los Angeles FC,"RCB, CB",29,5000000,2025-06-30,7,211,1,...,NaN,1.28,NaN,NaN,0.00,0.00,0.0,0.0,0,0
1,M. Robinson,Cincinnati,Cincinnati,"CB, RCB",27,5000000,2025-12-31,28,2803,1,...,NaN,1.41,NaN,NaN,0.00,0.00,0.0,0.0,0,0
2,J. Glesnes,Philadelphia Union,Philadelphia Union,RCB,30,4000000,2025-12-31,31,3056,2,...,NaN,0.94,NaN,NaN,0.00,0.00,0.0,0.0,0,0
3,J. Elliott,Chicago Fire,Philadelphia Union,"LCB, CB",29,3500000,NaN,27,2571,0,...,NaN,0.77,NaN,NaN,0.04,0.04,100.0,0.0,0,0
4,W. Zimmerman,Nashville SC,Nashville SC,"RCB, CB",31,3500000,2025-12-31,25,2126,1,...,NaN,2.54,NaN,NaN,0.00,0.00,0.0,0.0,0,0


In [3]:
# THIS ONLY WORKS WITH THE MLS DATASET (obv), AND THIS AIMS TO DEMONSTRATE THE PRESENCE OF PLAYERS WHOSE MAIN POSITION IS NOT A CENTRAL DEFENDER LIKE SERGIO BUSQUETS, HENCE WHY WE NEED TO DO THE FILTERING BELOW
bruh = df[df['Team within selected timeframe']=='Inter Miami']
bruh.head()

,Player,Team,Team within selected timeframe,Position,Age,Market value,Contract expires,Matches played,Minutes played,Goals,...,Prevented goals per 90,Back passes received as GK per 90,Exits per 90,Aerial duels per 90.1,Free kicks per 90,Direct free kicks per 90,"Direct free kicks on target, %",Corners per 90,Penalties taken,"Penalty conversion, %"
20,N. Freire,Independiente,Inter Miami,LCB,31,2500000,2025-06-30,10,816,0,...,NaN,2.98,NaN,NaN,0.00,0.0,0.0,0.0,0,0
36,Sergio Busquets,Inter Miami,Inter Miami,"DMF, LCB, CB",36,2000000,2025-12-31,32,2924,1,...,NaN,0.58,NaN,NaN,0.03,0.0,0.0,0.0,0,0
41,D. Martínez,Inter Miami,Inter Miami,LCB,26,1500000,2026-12-31,7,615,1,...,NaN,1.02,NaN,NaN,0.00,0.0,0.0,0.0,0,0
51,S. Kryvtsov,NaN,Inter Miami,"LCB, CB, RCB",34,1500000,NaN,23,1559,1,...,NaN,2.83,NaN,NaN,0.00,0.0,0.0,0.0,0,0
75,N. Allen,Inter Miami II,Inter Miami,"LCB, LB",20,1000000,NaN,20,1162,0,...,NaN,1.39,NaN,NaN,0.00,0.0,0.0,0.0,0,0


In [4]:
# Filtering function
def filter_by_exact_positions(df, positions):
    pattern = r'\b(?:' + '|'.join(positions) + r')\b'
    main_pos = df['Position'].str.split(',').str[0]  # get the main position before comma
    return df[main_pos.str.contains(pattern, case=False, na=False, regex=True)]

filtered_df = filter_by_exact_positions(df, ['CB','RCB','LCB']) # central defenders
filtered_df = filtered_df.rename(columns={'Team within selected timeframe': 'team during selection'})
filtered_df = filtered_df.rename(columns={'Team': 'Current team'})  # Changing current team column to squad just because I prefer it to be like that
filtered_df = filtered_df[filtered_df['Minutes played']>=1500] #you can adjust this but the article sets 1500 minutes for
filtered_df.columns = filtered_df.columns.str.lower()
filtered_df.head()

,player,current team,team during selection,position,age,market value,contract expires,matches played,minutes played,goals,...,prevented goals per 90,back passes received as gk per 90,exits per 90,aerial duels per 90.1,free kicks per 90,direct free kicks per 90,"direct free kicks on target, %",corners per 90,penalties taken,"penalty conversion, %"
1,M. Robinson,Cincinnati,Cincinnati,"CB, RCB",27,5000000,2025-12-31,28,2803,1,...,NaN,1.41,NaN,NaN,0.00,0.00,0.0,0.0,0,0
2,J. Glesnes,Philadelphia Union,Philadelphia Union,RCB,30,4000000,2025-12-31,31,3056,2,...,NaN,0.94,NaN,NaN,0.00,0.00,0.0,0.0,0,0
3,J. Elliott,Chicago Fire,Philadelphia Union,"LCB, CB",29,3500000,NaN,27,2571,0,...,NaN,0.77,NaN,NaN,0.04,0.04,100.0,0.0,0,0
4,W. Zimmerman,Nashville SC,Nashville SC,"RCB, CB",31,3500000,2025-12-31,25,2126,1,...,NaN,2.54,NaN,NaN,0.00,0.00,0.0,0.0,0,0
5,A. Malanda,Charlotte FC,Charlotte FC,RCB,23,3000000,2026-12-31,34,3447,1,...,NaN,1.59,NaN,NaN,0.03,0.03,0.0,0.0,0,0


In [5]:
#So uhhh, this is where I do the Z-score normalization

from scipy.stats import zscore

# Columns to exclude from z-score conversion
exclude_columns = ['player', 'current team', 'conference', 'team during selection', 'position', 'age', 'market value', 'contract expires',
                   'matches played', 'minutes played', 'passport country', 'foot', 'height']

# Columns to apply z-score conversion
columns_to_zscore = [col for col in filtered_df.columns if col not in exclude_columns and filtered_df[col].dtype in ['int64', 'float64']]

# Applying z-score normalization safely
filtered_df.loc[:, columns_to_zscore] = filtered_df[columns_to_zscore].apply(lambda x: zscore(x, nan_policy='omit')).fillna(0)

# Columns to display
columns_to_display = ['player', 'current team', 'team during selection', 'position', 'successful defensive actions per 90', 'defensive duels per 90',
                      'defensive duels won, %', 'aerial duels per 90', 'aerial duels won, %', 'interceptions per 90', 'padj interceptions']

# Displaying the first few rows to see how it looks like after this stage
filtered_df[columns_to_display].head()


<ipython-input-5-e310d8d6bb06>:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[ 0.26277607  1.38303194 -0.8574798   0.26277607  0.26277607 -0.8574798
 -0.8574798  -0.8574798   0.26277607 -0.8574798   0.26277607  0.26277607
 -0.8574798   0.26277607  0.26277607 -0.8574798   0.26277607  0.26277607
  2.50328782 -0.8574798   1.38303194  1.38303194 -0.8574798   0.26277607
  2.50328782  2.50328782  1.38303194  1.38303194 -0.8574798   1.38303194
  0.26277607 -0.8574798   1.38303194 -0.8574798  -0.8574798  -0.8574798
  2.50328782 -0.8574798  -0.8574798   0.26277607  0.26277607 -0.8574798
 -0.8574798  -0.8574798   1.38303194 -0.8574798  -0.8574798  -0.8574798
 -0.8574798   0.26277607 -0.8574798  -0.8574798  -0.8574798  -0.8574798
  1.38303194  0.26277607 -0.8574798  -0.8574798   0.26277607  0.26277607
  0.26277607  0.26277607  0.26277607  0.26277607 -0.8574798   1.38303194
  0.26277607 -0.8574798   1.38303194 -0.8574798  

,player,current team,team during selection,position,successful defensive actions per 90,defensive duels per 90,"defensive duels won, %",aerial duels per 90,"aerial duels won, %",interceptions per 90,padj interceptions
1,M. Robinson,Cincinnati,Cincinnati,"CB, RCB",0.003822,0.291611,1.004822,1.214534,2.201220,-0.630016,0.134076
2,J. Glesnes,Philadelphia Union,Philadelphia Union,RCB,0.226354,-0.175197,0.133501,-0.446843,1.102512,0.314735,0.223185
3,J. Elliott,Chicago Fire,Philadelphia Union,"LCB, CB",0.797196,0.690520,-0.428145,-0.285020,0.395411,0.565684,0.356848
4,W. Zimmerman,Nashville SC,Nashville SC,"RCB, CB",0.913299,-0.285534,1.118306,1.160593,0.998166,1.348055,0.691007
5,A. Malanda,Charlotte FC,Charlotte FC,RCB,-0.218709,-1.278563,-0.283886,-0.004528,-0.984419,1.392340,0.100660


In [6]:
print(filtered_df.columns.duplicated().any())  # True means you have duplicates - was directly taken from Henrik's code

False


In [7]:
# This is the min-max feature scaling step

# Function to normalize the data to a [0, 1] range
def NormalizeData(data):
    # Subtract the minimum value from each element and divide by the range
    return (data - np.min(data)) / (np.max(data) - np.min(data))

# Ensure you're working with a copy of the original DataFrame
# This avoids accidental changes to the original data and avoids the SettingWithCopyWarning
filtered_df = filtered_df.copy()

# Iterate over the specified columns (columns_to_zscore) to normalize them
for column in columns_to_zscore:
    # Step 1: Shift the values in the column to ensure no negative values
    # Add the absolute value of the column's minimum to all values in the column
    # This ensures the minimum value becomes 0
    filtered_df[column] = filtered_df[column] + abs(filtered_df[column].min())

    # Step 2: Normalize the column to a [0, 1] range
    # Apply the NormalizeData function to scale the column values
    filtered_df[column] = NormalizeData(filtered_df[column])

filtered_df[columns_to_display].head()

,player,current team,team during selection,position,successful defensive actions per 90,defensive duels per 90,"defensive duels won, %",aerial duels per 90,"aerial duels won, %",interceptions per 90,padj interceptions
1,M. Robinson,Cincinnati,Cincinnati,"CB, RCB",0.530973,0.498054,0.695668,0.683036,1.000000,0.357143,0.524038
2,J. Glesnes,Philadelphia Union,Philadelphia Union,RCB,0.581858,0.391051,0.530761,0.339286,0.772380,0.564935,0.543269
3,J. Elliott,Chicago Fire,Philadelphia Union,"LCB, CB",0.712389,0.589494,0.424463,0.372768,0.625890,0.620130,0.572115
4,W. Zimmerman,Nashville SC,Nashville SC,"RCB, CB",0.738938,0.365759,0.717146,0.671875,0.750763,0.792208,0.644231
5,A. Malanda,Charlotte FC,Charlotte FC,RCB,0.480088,0.138132,0.451766,0.430804,0.340031,0.801948,0.516827


# Central Defenders Attributed Weightings
## Anchor
- Plays safe passes and does his defending close to goal, especially with clearances and blocks. Ruben Dias, Matthijs de Ligt, Milan Skriniar.

## Spreader
- Plays longer, more direct passes. Will sometimes step out in possession or to close down in defence. Marquinhos, Eder Militao, Jules Kounde
- Heavier weights on long passes / 90, accurate long passes, %, and long passes completions.

## Aggressor
- Likes to tackle and dribble. Not big on long passes or clearances. Includes a lot of outside centre-backs. Alessandro Bastoni, Lucas Hernandez, Ronald Araujo


In [8]:
#Central Defender Specific Attributed Weighing Step
# Create new columns in a batch
new_columns = {
    # Crossing
    'crosses_rating': filtered_df['crosses per 90'] * 0.3,
    'crossing_accuray_rating': filtered_df['accurate crosses, %'] * 0.4,
    'deep completed crosses_rating': filtered_df['deep completed crosses per 90'] * 0.3,
    # 'deep_completions_rating': filtered_df['deep completions per 90'] * 0.1,
    'crossing_att': filtered_df['crosses per 90'] * 0.3 + filtered_df['accurate crosses, %'] * 0.4 +filtered_df['deep completed crosses per 90'] * 0.3,

    # Dribbling
    'dribbles_rating': filtered_df['dribbles per 90'] * 0.3,
    'dribbling_success_rating': filtered_df['successful dribbles, %'] * 0.3,
    'dribbles_progressiverun_rating':filtered_df['progressive runs per 90'] * 0.4,
    'dribbling_att': filtered_df['dribbles per 90'] * 0.3 + filtered_df['successful dribbles, %'] * 0.3 + filtered_df['progressive runs per 90'] * 0.4,

    #Anchor - back passes, shots blocked
    'back_passes_rating': filtered_df['back passes per 90'] * 0.1,
    'back_passes_accuracy_rating': filtered_df['accurate back passes, %'] * 0.4,
    'shots_blocked_rating': filtered_df['shots blocked per 90'] * 0.5,
    'anchor_att': filtered_df['back passes per 90'] * 0.1 + filtered_df['accurate back passes, %'] * 0.4 + filtered_df['shots blocked per 90'] * 0.5,

    #General Passing
    'lateral_passes_rating': filtered_df['lateral passes per 90'] * 0.2,
    'lateral_passes_accuracy_rating': filtered_df['accurate lateral passes, %'] * 0.1,
    'short_medium_passes_rating': filtered_df['short / medium passes per 90'] * 0.2,
    'short_medium_passes_accuracy_rating': filtered_df['accurate short / medium passes, %'] * 0.1,
    'long_passes_rating': filtered_df['long passes per 90'] * 0.25,
    'accurate_long_passes_rating': filtered_df['accurate long passes, %'] * 0.15,
    'general_pass_att': filtered_df['lateral passes per 90'] * 0.2 + filtered_df['accurate lateral passes, %'] * 0.1 + filtered_df['short / medium passes per 90'] * 0.2 + filtered_df['accurate short / medium passes, %'] * 0.1 + filtered_df['long passes per 90'] * 0.25 + filtered_df['accurate long passes, %'] * 0.15,

    #Chance creation
    'progressive_passes_rating': filtered_df['progressive passes per 90'] * 0.2,
    'accuracy_progressive_passes__rating': filtered_df['accurate progressive passes, %'] * 0.05,
    'through_passes_rating': filtered_df['through passes per 90'] * 0.2,
    'accurate_through_passes_rating': filtered_df['accurate through passes, %'] * 0.05,
     'passes_to_penaltyarea_rating': filtered_df['passes to penalty area per 90'] * 0.2,
     'accurate_passes_to_penaltyarea%_rating': filtered_df['accurate passes to penalty area, %'] * 0.1,
    'expected_assist_rating': filtered_df['xa per 90'] * 0.25,

    'chance_creation_att': filtered_df['progressive passes per 90'] * 0.2 +filtered_df['accurate progressive passes, %'] * 0.05 + filtered_df['through passes per 90'] * 0.2 +filtered_df['accurate through passes, %'] * 0.05 + filtered_df['passes to penalty area per 90'] * 0.2 +filtered_df['accurate passes to penalty area, %'] * 0.1 + filtered_df['xa per 90'] * 0.25,

    # Defending
    'DefensiveDuelRating': filtered_df['defensive duels per 90'] * 0.25,
    'DefensiveDuelsWonRating': filtered_df['defensive duels won, %'] * 0.15,
    'Aerial Rating': filtered_df['aerial duels per 90'] * 0.2,
    'Aerial Duels Won Rating': filtered_df['aerial duels won, %'] * 0.1,
    'interceptions_rating': filtered_df['interceptions per 90'] * 0.05,
    'adjusted_interceptions_rating' : filtered_df['padj interceptions']*0.15,
    'sliding tackles_rating': filtered_df['sliding tackles per 90'] * 0.025,
    'adjusted_sliding tackles_rating' : filtered_df['padj sliding tackles']*0.075,
    'defensive_att': filtered_df['defensive duels per 90'] * 0.25 +
                     filtered_df['defensive duels won, %'] * 0.15 +
                     filtered_df['aerial duels per 90'] * 0.2 +
                     filtered_df['aerial duels won, %'] * 0.1 +
                     filtered_df['interceptions per 90'] * 0.05 +
                     filtered_df['padj interceptions']*0.15 +
                     filtered_df['sliding tackles per 90'] * 0.025 +
                     filtered_df['padj sliding tackles']*0.075,

     # Ground Defending
    'DefensiveDuelRating': filtered_df['defensive duels per 90'] * 0.3,
    'DefensiveDuelsWonRating': filtered_df['defensive duels won, %'] * 0.2,
    'interceptions_rating': filtered_df['interceptions per 90'] * 0.1,
    'adjusted_interceptions_rating' : filtered_df['padj interceptions']*0.2,
    'sliding tackles_rating': filtered_df['sliding tackles per 90'] * 0.075,
    'adjusted_sliding tackles_rating' : filtered_df['padj sliding tackles']*0.125,

    'ground_defense_att': filtered_df['defensive duels per 90'] * 0.3 +
                          filtered_df['defensive duels won, %'] * 0.2 +
                          filtered_df['interceptions per 90'] * 0.1 +
                          filtered_df['padj interceptions']*0.2 +
                          filtered_df['sliding tackles per 90'] * 0.075 +
                          filtered_df['padj sliding tackles']*0.125,

    #Aggressor
    'DefensiveDuelRating': filtered_df['defensive duels per 90'] * 0.3,
    'DefensiveDuelsWonRating': filtered_df['defensive duels won, %'] * 0.2,
    'interceptions_rating': filtered_df['interceptions per 90'] * 0.1,
    'adjusted_interceptions_rating': filtered_df['padj interceptions'] * 0.15,
    'sliding tackles_rating': filtered_df['sliding tackles per 90'] * 0.1,
    'adjusted_sliding tackles_rating': filtered_df['padj sliding tackles'] * 0.15,

    'aggressor_att': filtered_df['defensive duels per 90'] * 0.3 +
                    filtered_df['defensive duels won, %'] * 0.2 +
                    filtered_df['interceptions per 90'] * 0.1 +
                    filtered_df['padj interceptions'] * 0.15 +
                    filtered_df['sliding tackles per 90'] * 0.1 +
                    filtered_df['padj sliding tackles'] * 0.15,


    # Aerial Ability
    'aerial_duel_rating': filtered_df['aerial duels per 90'] * 0.5,
    'aerial_duels_won_rating': filtered_df['aerial duels won, %'] * 0.5,

    'aerial_att': filtered_df['aerial duels per 90'] * 0.5 + filtered_df['aerial duels won, %'] * 0.5
}

# Add all new columns to the DataFrame at once
filtered_df = pd.concat([filtered_df, pd.DataFrame(new_columns)], axis=1)

# Optionally: Create a fresh copy of the DataFrame to de-fragment
filtered_df = filtered_df.copy()


In [9]:
# Creating the new DataFrame 'ratings' with the specified columns
ratings_columns = [
    "player",
    'current team',
    'team during selection',
    "minutes played",
    "crossing_att", "dribbling_att","general_pass_att", "chance_creation_att", "defensive_att","ground_defense_att","aggressor_att", "aerial_att",'anchor_att']

ratings = filtered_df[ratings_columns].copy()

In [10]:
# Second min-max scaling - the one after attributed weighing

def NormalizeData(data):
    # Subtract the minimum value from each element and divide by the range
    return (data - np.min(data)) / (np.max(data) - np.min(data))

# Ensure a copy of the DataFrame to prevent unintended changes to the original data
ratings = ratings.copy()

# Get the column names from the 5th column onwards
columns_to_normalize = ratings.columns[5:]  # Index starts at 0, so 5 corresponds to the 6th column

# Iterate over the selected columns and apply normalization
for column in columns_to_normalize:
    # Step 1: Shift values to ensure no negative numbers
    # Add the absolute minimum of the column to all its values
    ratings[column] = ratings[column] + abs(ratings[column].min())

    # Step 2: Normalize the column to a [0, 1] range
    # Apply the NormalizeData function to scale the column
    ratings[column] = NormalizeData(ratings[column])

In [11]:
ratings.head()

,player,current team,team during selection,minutes played,crossing_att,dribbling_att,general_pass_att,chance_creation_att,defensive_att,ground_defense_att,aggressor_att,aerial_att,anchor_att
1,M. Robinson,Cincinnati,Cincinnati,2803,0.015175,0.344049,0.623092,0.552814,0.824938,0.631027,0.574511,0.992285,0.296017
2,J. Glesnes,Philadelphia Union,Philadelphia Union,3056,0.318996,0.618625,0.497960,0.386653,0.574935,0.642414,0.608800,0.654071,0.475711
3,J. Elliott,Chicago Fire,Philadelphia Union,2571,0.000000,0.269603,0.644462,0.367863,0.618362,0.692116,0.638718,0.587177,0.526900
4,W. Zimmerman,Nashville SC,Nashville SC,2126,0.119844,0.462855,0.840124,0.452312,0.792094,0.721157,0.653209,0.838146,0.538771
5,A. Malanda,Charlotte FC,Charlotte FC,3447,0.165041,0.323509,0.744067,0.320320,0.281131,0.322456,0.265021,0.452320,0.563087


In [12]:
#remove duplicates
ratings = ratings.loc[:, ~ratings.columns.duplicated(keep='last')]

In [13]:
# Make a copy of the ratings DataFrame to avoid accidental modifications
ratings = ratings.copy()

# Multiply all columns from the 6th column onwards by 100
ratings.iloc[:, 3:] = ratings.iloc[:, 3:] * 100  # Scale 6th column onward by 100

# Round the same columns to 1 decimal place
ratings.iloc[:, 3:] = ratings.iloc[:, 3:].round(2)  # Round to 1 decimal place

# Reset the index of the sorted DataFrame to ensure a clean, sequential index
ratings.reset_index(drop=True, inplace=True)

ratings.index = ratings.index + 1
ratings.head(50)

,player,current team,team during selection,minutes played,crossing_att,dribbling_att,general_pass_att,chance_creation_att,defensive_att,ground_defense_att,aggressor_att,aerial_att,anchor_att
1,M. Robinson,Cincinnati,Cincinnati,280300,1.52,34.40,62.31,55.28,82.49,63.10,57.45,99.23,29.60
2,J. Glesnes,Philadelphia Union,Philadelphia Union,305600,31.90,61.86,49.80,38.67,57.49,64.24,60.88,65.41,47.57
3,J. Elliott,Chicago Fire,Philadelphia Union,257100,0.00,26.96,64.45,36.79,61.84,69.21,63.87,58.72,52.69
4,W. Zimmerman,Nashville SC,Nashville SC,212600,11.98,46.29,84.01,45.23,79.21,72.12,65.32,83.81,53.88
5,A. Malanda,Charlotte FC,Charlotte FC,344700,16.50,32.35,74.41,32.03,28.11,32.25,26.50,45.23,56.31
6,L. Abram,Atlanta United,Atlanta United,169000,1.28,23.83,63.97,34.18,2.12,2.65,1.47,41.70,61.03
7,K. Miller,Portland Timbers,Portland Timbers,224800,1.40,57.61,65.99,25.10,49.62,55.69,62.10,74.54,58.36
8,J. Glad,Real Salt Lake,Real Salt Lake,275000,0.00,37.86,75.85,2.07,64.05,61.91,51.76,65.31,58.50
9,M. Amundsen,Columbus Crew,Columbus Crew,259300,43.69,72.26,62.64,79.07,51.16,62.46,53.10,38.02,41.60
10,M. Miazga,Cincinnati,Cincinnati,157100,0.00,38.23,80.94,28.24,83.84,81.08,74.89,93.54,28.75


In [14]:
# Weight the overall attributes in terms of importance
ratings["Spreader"] = ratings['general_pass_att'] * 0.3 + ratings['ground_defense_att'] * 0.2 + ratings['chance_creation_att'] * 0.2 + ratings['aerial_att']*0.05 + ratings['dribbling_att']*0.1 + ratings['crossing_att']*0.15
ratings["Aggressor"] = ratings['aggressor_att'] * 0.4 + ratings['chance_creation_att'] * 0.1 + ratings['aerial_att']*0.1 + ratings['dribbling_att']*0.2 + ratings['crossing_att']*0.1 + ratings['general_pass_att'] * 0.1
ratings["Anchor"] = ratings['anchor_att'] * 0.4 + ratings['aerial_att']*0.2 + ratings['dribbling_att']*0.05 + ratings['crossing_att']*0.05 + ratings['general_pass_att'] * 0.1 + ratings['ground_defense_att'] * 0.2
ratings["CB_Score"] = (ratings['Spreader'] + ratings['Aggressor'] + ratings['Anchor'])/3
ratings = ratings.sort_values(by='CB_Score', ascending=False).reset_index(drop=True)
ratings.index = ratings.index + 1

ratings.head(50)

,player,current team,team during selection,minutes played,crossing_att,dribbling_att,general_pass_att,chance_creation_att,defensive_att,ground_defense_att,aggressor_att,aerial_att,anchor_att,Spreader,Aggressor,Anchor,CB_Score
1,M. Chanot,Los Angeles FC,Los Angeles FC,187000,41.89,40.77,83.22,28.45,89.39,94.55,92.06,82.86,71.47,64.0695,68.620,76.5250,69.738167
2,Y. Cheberko,Columbus Crew,Columbus Crew,211200,13.03,67.79,94.44,46.49,80.90,89.04,80.50,63.13,62.27,67.3280,67.467,68.8270,67.874000
3,G. Campbell,CF Montréal,CF Montréal,253600,13.38,99.18,64.24,60.82,72.41,78.50,70.62,63.88,74.51,62.2550,68.316,70.3320,66.967667
4,R. Czichos,NaN,Chicago Fire,264600,12.75,86.51,77.41,47.20,74.79,85.83,82.77,62.74,58.96,63.5295,70.420,66.0020,66.650500
5,S. Kryvtsov,NaN,Inter Miami,155900,42.27,58.05,77.92,26.64,63.81,83.48,73.74,47.81,100.00,59.9360,60.570,79.0660,66.524000
6,A. Souquet,Chicago Fire II,Chicago Fire,208000,41.40,98.02,41.28,87.97,65.29,87.26,84.81,37.28,27.25,65.3060,74.321,46.9070,62.178000
7,Andreu Fontàs,NaN,Sporting KC,155700,22.97,53.72,99.93,60.77,64.18,76.66,70.15,58.82,30.46,69.2235,63.053,53.1075,61.794667
8,N. Eile,New York RB,New York RB,291200,12.18,89.67,85.35,64.25,72.26,61.16,56.17,79.93,31.14,65.4775,64.573,54.3015,61.450667
9,J. Waterman,CF Montréal,CF Montréal,246600,3.14,54.76,100.00,30.74,53.55,84.03,75.22,34.81,68.85,60.6415,57.909,64.2030,60.917833
10,H. Kessler,St. Louis City,St. Louis City,233200,41.51,42.50,65.74,6.39,88.07,95.18,86.09,62.40,63.97,53.6325,60.540,67.8785,60.683667


In [15]:
spreader_rate = ratings.copy()
columns_to_display = ["player","current team","team during selection","minutes played","Spreader"]
spreader_rate = spreader_rate[columns_to_display]
spreader_rate = spreader_rate.sort_values(by='Spreader', ascending=False).reset_index(drop=True)
spreader_rate.index = spreader_rate.index + 1
spreader_rate.head(50)


,player,current team,team during selection,minutes played,Spreader
1,Andreu Fontàs,NaN,Sporting KC,155700,69.2235
2,Y. Cheberko,Columbus Crew,Columbus Crew,211200,67.3280
3,N. Eile,New York RB,New York RB,291200,65.4775
4,A. Souquet,Chicago Fire II,Chicago Fire,208000,65.3060
5,M. Chanot,Los Angeles FC,Los Angeles FC,187000,64.0695
6,R. Czichos,NaN,Chicago Fire,264600,63.5295
7,M. Amundsen,Columbus Crew,Columbus Crew,259300,62.7785
8,T. Avilés,Inter Miami,Inter Miami,268200,62.5455
9,G. Campbell,CF Montréal,CF Montréal,253600,62.2550
10,J. Waterman,CF Montréal,CF Montréal,246600,60.6415


In [16]:
anchor_rate = ratings.copy()
columns_to_display = ["player","current team","team during selection","minutes played","Anchor"]
anchor_rate = anchor_rate[columns_to_display]
anchor_rate = anchor_rate.sort_values(by='Anchor', ascending=False).reset_index(drop=True)
anchor_rate.index = anchor_rate.index + 1
anchor_rate.head(50)


,player,current team,team during selection,minutes played,Anchor
1,S. Kryvtsov,NaN,Inter Miami,155900,79.0660
2,M. Chanot,Los Angeles FC,Los Angeles FC,187000,76.5250
3,G. Campbell,CF Montréal,CF Montréal,253600,70.3320
4,Y. Cheberko,Columbus Crew,Columbus Crew,211200,68.8270
5,H. Kessler,St. Louis City,St. Louis City,233200,67.8785
6,K. Long,Toronto,Toronto,238400,66.4800
7,R. Czichos,NaN,Chicago Fire,264600,66.0020
8,J. Waterman,CF Montréal,CF Montréal,246600,64.2030
9,W. Zimmerman,Nashville SC,Nashville SC,212600,64.0525
10,S. Gregersen,Atlanta United,Atlanta United,221300,63.8025


In [17]:
aggressor_rate = ratings.copy()
columns_to_display = ["player","current team","team during selection","minutes played","Aggressor"]
aggressor_rate = aggressor_rate[columns_to_display]
aggressor_rate = aggressor_rate.sort_values(by='Aggressor', ascending=False).reset_index(drop=True)
aggressor_rate.index = aggressor_rate.index + 1
aggressor_rate.head(50)


,player,current team,team during selection,minutes played,Aggressor
1,A. Souquet,Chicago Fire II,Chicago Fire,208000,74.321
2,R. Czichos,NaN,Chicago Fire,264600,70.420
3,M. Chanot,Los Angeles FC,Los Angeles FC,187000,68.620
4,G. Campbell,CF Montréal,CF Montréal,253600,68.316
5,Y. Cheberko,Columbus Crew,Columbus Crew,211200,67.467
6,T. Avilés,Inter Miami,Inter Miami,268200,65.366
7,N. Eile,New York RB,New York RB,291200,64.573
8,X. Arreaga,Barcelona,New England,242100,64.204
9,Andreu Fontàs,NaN,Sporting KC,155700,63.053
10,D. Padelford,Minnesota United,Minnesota United,179700,61.335
